# Read in a csv file from TEPCat
# and allow some simple comparisons.

In [1]:
% matplotlib inline
from matplotlib import rcParams
#rcParams['figure.figsize'] = 8, 6
from matplotlib.font_manager import FontProperties
import scipy.integrate as integrate
import csv
import math
import numpy as np
import pylab as p
import pickle
#import my_fns

# For magnitude lookups
from astroquery.irsa import Irsa
import astropy.units as u
import astropy.constants as cs

# For TESS
#import tess_stars2px as tesspt

from exo_fns import *

Loading BokehJS ...

/home/jacob/anaconda2/lib/python2.7/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '


# Download new tables

In [2]:
download_tepcat()

Downloaded system parameters, "Well-studied transiting planets"
Downloaded magnitudes, "For planning observations"


In [3]:
#update_TESS() # Currently bugged, use tables on Slack

Ehrenreich+14 staring mode GJ3470,
follow up done in spatial scanning by Tsiaras(?)

## Basic Planet params

In [4]:
od = get_tepcat_planets()
od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=0)
od['SH'] = calc_scale_h(od['TEQ'], od['g'])
od['ATMDEPTH'] = calc_transit_depth(od['SH'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value)
od['RLDEPTH'] = calc_transit_depth(od['R_b']*cs.R_jup.value*3, od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value)
od['EDR'] = reflected_depth(od['R_b']*cs.R_jup.value, od['a(AU)']*cs.au.value, A=0.5)

# Pickle save/load
#with open('od.pic', 'w') as g:
#    pickle.dump(od, g)

#with open('od.pic', 'r') as g:
#    od = pickle.load(g)

exo_fns.py:315: RuntimeWarning: divide by zero encountered in divide
  return np.sqrt(np.divide(Rs,a)/f_redist)*Ts*(2**0.25) * (1-A)**0.25
exo_fns.py:315: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.divide(Rs,a)/f_redist)*Ts*(2**0.25) * (1-A)**0.25
exo_fns.py:326: RuntimeWarning: divide by zero encountered in divide
  return cs.k_B.value*T/g/(mu*cs.m_p.value)
exo_fns.py:433: RuntimeWarning: divide by zero encountered in divide
  return (Rp/a/2)**2 *A


In [7]:
print_planet(od, '55_Cnc_e')
print_planet(od, 'KELT-09')

Cnc55e = get_planet(od, '55_Cnc_e')
KELT09 = get_planet(od, 'KELT-09')

--------
55_Cnc_e
--------
                M_b 	0.0243
                R_b 	0.1692
                g_b 	21.4
                Teq 	2349.0
                TEQ 	2331.309064561629
                 SH 	762224.4157281633
               Vmag 	5.95
               Kmag 	4.01
              a(AU) 	0.01544
             Period 	0.737
                  e 	0.05
                M_A 	0.873
                R_A 	0.954
                  g 	21.038806707912926
               Teff 	5172.0
             [Fe/H] 	0.35
                 RA 	133.15054166666664
                Dec 	28.33138888888889
           ATMDEPTH 	4.186295694446746e-05
              loggA 	4.43
Discovery_reference 	2011ApJ...737L..18W
              rho_b 	4.71
              rho_A 	1.0
   Recent_reference 	arXiv:1908.06299
                EDR 	3.428310964868279e-06
            RLDEPTH 	0.001993086305194623
-------
KELT-09
-------
                M_b 	2.88
                R_b 	1.936
                g_b 	19.0
                Teq 	3921.0
         

### Noise Scalings

In [8]:
noise_scalings = {'WFC3_G141': 
                      {'Transit': {'System':'GJ_1214', 'Noise':70e-6 , 'Ref':'Kreidberg+14'},
                       'Eclipse': {'System':'GJ_1214', 'Noise':70e-6 , 'Ref':'Kreidberg+14'}},
                  'WFC3_G102': 
                      {'Transit': {'System':'WASP-012', 'Noise':164e-6 , 'Ref':'?'},
                       'Eclipse': {'System':'WASP_012', 'Noise':164e-6 , 'Ref':'?'}},
                  'STIS': 
                      {'Transit': {'System': 'HD_209458', 'Noise':52e-6, 'Ref':'Charbonneau+02'},
                       'Eclipse': {'System': 'HD_209458', 'Noise':110e-6, 'Ref':'Brown+01'}},
                  'Spitzer':
                      {'Eclipse': {'System': 'WASP-018', 'Noise':100e-6, 'Ref':'Maxted+14'}
                        }
                  }

# Transmission

## Transit depths

In [10]:
def get_transit_scaling(instrument, scale_band, noise_scalings, plot=False):

    od = get_tepcat_planets()

    od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
    od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=0)
    od['SH'] = calc_scale_h(od['TEQ'], od['g'])
    od['ATMDEPTH'] = calc_transit_depth(od['SH'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value)
    
    
    scale = noise_scalings[instrument]['Transit']
    scale_ind = list(od['System']).index(scale['System'])
    scale_N = scale['Noise']
    scale_mag = od[scale_band][scale_ind]
    
    
    od['TSNR'] = (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['ATMDEPTH'] / scale_N
    
    od['ATMDEPTH'] *= 1e6 # for visualization
    table_name = './tables/{}/transit_snr_{}.csv'.format(instrument, scale_band[0])
    csv_arrays(table_name, od, 
       keys=['System', scale_band, 'ATMDEPTH', 'TSNR', 'Period', 'M_b', 'g', 'TEQ'],
       names=['Name', scale_band, 'AtmDepth','SNR', 'Period', 'Mass', 'logg', 'Eq Temp'],
       units = ['-', '-', 'ppm', '-', 'days', 'M_jup', '-', 'K'],
       formats=['{}','{:.2f}', '{:.1f}', '{:.3f}', '{:.2f}', '{:.2f}', '{:.2f}', '{:.0f}'],
       sort='TSNR', reverse=True, finite=True)
    if plot:
        afile = open(table_name, 'r+')
        csvReader1 = csv.reader(afile)
        lines = [ csvReader1.next() for _ in range(22) ]
        for line in lines:
            for val in line:
                print '{:<12s}'.format(val),
            print 
    
    ymax = np.max(od['TSNR'][np.isfinite(od['TSNR'])])
    plot_html(od, xname='M_b', yname='TSNR', x_range=(-0.1,3), y_range=(0,ymax), 
        title='Transmission, {}-band with {}'.format(scale_band[0], instrument), 
        fname='./tables/{}/transit_snr_{}.html'.format(instrument, scale_band[0]), plot=plot)
    od['ATMDEPTH'] /= 1e6
    
    return od

#### Example

In [11]:
# WFC3 G141
od = get_transit_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True)

Name         Kmag         AtmDepth     SNR          Period       Mass         logg         Eq Temp     
-            -            ppm          -            days         M_jup        -            K           
HD_189733    5.54         272.3        17.299       2.22         1.15         21.52        1417        
HD_209458    6.31         387.9        17.282       3.52         0.71         9.29         1735        
WASP-069     7.46         643.6        16.886       3.87         0.26         5.77         1142        
WASP-107     8.64         1102.5       16.799       5.72         0.12         3.45         876         
KELT-11      6.12         330.2        16.057       4.74         0.17         2.33         2026        
WASP-127     8.64         1033.4       15.746       4.18         0.18         2.38         1705        
GJ_1214      8.78         992.4        14.177       1.58         0.02         7.57         679         
WASP-094     8.87         675.3        9.256        3.95        

exo_fns.py:292: RuntimeWarning: divide by zero encountered in divide
  return np.sqrt(np.divide(Rs,a)/f_redist)*Ts*(2**0.25) * (1-A)**0.25
exo_fns.py:292: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.divide(Rs,a)/f_redist)*Ts*(2**0.25) * (1-A)**0.25
exo_fns.py:303: RuntimeWarning: divide by zero encountered in divide
  return cs.k_B.value*T/g/(mu*cs.m_p.value)
/home/jacob/anaconda2/lib/python2.7/site-packages/bokeh/io.py:423: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/jacob/anaconda2/lib/python2.7/site-packages/bokeh/io.py:433: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, u

In [ ]:
# WFC3 G141
od = get_transit_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True)

In [ ]:
# STIS (scaled to Charbennau+02)
od = get_transit_scaling('STIS', 'Vmag', noise_scalings, plot=True)

# Emission

In [21]:
def get_emission_scaling(instrument, scale_band, noise_scalings, plot=False, Ag=0, reflection=False, printn=20):
        
    od = get_tepcat_planets()

    od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
    od['Ag'] = np.ones(len(od['System']))*Ag
    od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=Ag)

    band = scale_band[0] # letter V or K
    T = np.ones_like(od['TEQ'])*1000.
    od['EDE'] = eclipse_depth(T, od['Teff'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value, band=band)
    od['EDR'] = reflected_depth(od['R_b']*cs.R_jup.value, od['a(AU)']*cs.au.value, A=Ag)
    od['ED'] = od['EDE'] + od['EDR']
        
    scale = noise_scalings[instrument]['Eclipse']
    scale_ind = list(od['System']).index(scale['System'])
    scale_N = scale['Noise']
    scale_mag = od[scale_band][scale_ind]
    
    
    od['EESNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDE'] / scale_N
    od['ERSNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDR'] / scale_N
    od['ESNR'] =   (10**(-(od[scale_band]-scale_mag)/2.5/2.))*(od['ED']) / scale_N
    od['E_N'] =  scale_N / (10**(-(od[scale_band]-scale_mag)/2.5/2.))*1e6
        
    od['EDE'] *= 1e6; od['EDR'] *= 1e6; od['ED'] *= 1e6
    table_name = './tables/{}/emission_snr_{}_A-{}.csv'.format(instrument, scale_band[0], Ag)
    csv_arrays(table_name, od, 
       keys=['System', scale_band, 'EDE', 'EDR', 'E_N', 'Ag', 'TEQ', 'M_b'],
       names=['Name', scale_band, 'Depth (Em)', 'Depth (Rf)', 'Noise', 'Albedo', 'Eq Temp', 'Mass'],
       units = ['-', '-', 'ppm', 'ppm', 'ppm', '-', 'K', 'M_jup'],
       formats=['{}','{:.2f}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.2f}', '{:.0f}', '{:.2f}'],
       sort='EDE', reverse=True, finite=True)
    if plot:
        afile = open(table_name, 'r+')
        csvReader1 = csv.reader(afile)
        lines = [ csvReader1.next() for _ in range(printn+2) ]
        for line in lines:
            for val in line:
                print '{:<10s}'.format(val),
            print 
    
    ymax = np.max(od['ED'][np.isfinite(od['ED'])])
    plot_html(od, xname='M_b', yname='ED', x_range=(-0.1,13), y_range=(0,ymax), 
        title='Eclipse depth, {}-band with {}, Ag={}'.format(scale_band[0], instrument, Ag), 
        fname='./tables/{}/eclipse_snr_{}_A-{}.html'.format(instrument, scale_band[0], Ag), plot=plot)
    
    return od

In [22]:
# WFC3 G141 emission
od = get_emission_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True, Ag=0., printn=20)

Name       Kmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
LHS_6343   8.70       352.1      0.0        67.5       0.00       426        62.10     
HATS-71    11.73      334.5      0.0        272.3      0.00       726        0.45      
Kepler-045 12.89      216.8      0.0        464.6      0.00       936        0.50      
HATS-06    11.22      199.0      0.0        215.3      0.00       858        0.32      
GJ_1214    8.78       148.9      0.0        70.0       0.00       679        0.02      
WASP-080   8.35       147.4      0.0        57.4       0.00       981        0.56      
WTS-2      13.41      122.4      0.0        590.3      0.00       1826       1.12      
WASP-043   9.27       108.5      0.0        87.7       0.00       1714       2.03      
Qatar-2    10.62      107.3      0.0        163.3      0.00       1528       2.47      
Qatar-9    11.10      103.7     

In [20]:
# STIS reflection
od = get_emission_scaling('STIS', 'Vmag', noise_scalings, plot=True, Ag=.5)

Name       Vmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
KELT-09    7.56       123.8      85.2       105.5      0.50       4051       2.88      
WASP-033   8.18       31.3       120.0      140.4      0.50       2735       2.16      
WASP-018   9.27       12.1       98.0       231.9      0.50       2412       10.52     
WASP-121   10.52      16.4       153.4      412.5      0.50       2358       1.18      
WASP-189   6.64       4.9        21.8       69.1       0.50       2641       2.13      
HD_189733  7.68       0.0        38.3       111.5      0.50       1191       1.15      
WASP-012   11.69      33.9       188.2      707.0      0.50       2562       1.47      
WASP-019   12.31      9.9        212.6      940.6      0.50       2078       1.14      
WASP-103   12.39      30.3       197.5      975.9      0.50       2490       1.47      
HD_209458  7.65       0.0       

In [19]:
# STIS reflection
od = get_emission_scaling('STIS', 'Vmag', noise_scalings, plot=True, Ag=0.)

Name       Vmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
KELT-09    7.56       346.3      0.0        105.5      0.00       4817       2.88      
WASP-033   8.18       138.0      0.0        140.4      0.00       3252       2.16      
WASP-189   6.64       22.5       0.0        69.1       0.00       3141       2.13      
WASP-018   9.27       64.5       0.0        231.9      0.00       2868       10.52     
WASP-012   11.69      164.3      0.0        707.0      0.00       3046       1.47      
WASP-121   10.52      90.1       0.0        412.5      0.00       2804       1.18      
WASP-103   12.39      153.2      0.0        975.9      0.00       2961       1.47      
KELT-16    11.90      98.3       0.0        778.7      0.00       2917       2.75      
KELT-01    10.70      39.1       0.0        448.1      0.00       2882       27.38     
WASP-087   10.74      33.4      

exo_fns.py:433: RuntimeWarning: invalid value encountered in multiply
  return (Rp/a/2)**2 *A


In [ ]:
# Spitzer 4.5 micron emission
od = get_emission_scaling('WFC3_G141', 'Kmag', noise_scalings, plot=True, Ag=0.)

### TESS planets in order

In [ ]:
od = get_tepcat_planets(TESS=True)

first_sectors = []
for planet, sec in zip(od['System'], od['TESS_sectors']):
    if len(sec) > 0:
        first_sectors.append(13-min(sec))
    else:
        first_sectors.append(None)

_, _, i_TESSbest = zip(*sorted(zip(od['TESS_days'], first_sectors, np.arange(len(od['System']))), reverse=True))

with open('TESS_Sectors.txt', 'w') as g:
    g.write('System\tDays\tSectors\n')
    for i in i_TESSbest:
            if od['Dec'][i] < 0: 
                if od['TESS_days'][i] > 0:
                    line = '{:10s}\t{}\t{}\n'.format(od['System'][i], od['TESS_days'][i], od['TESS_sectors'][i])
                    g.write(line)
                    print line[:-1]

# Spitzer

In [34]:
def get_emission_band(instrument, band, scale_band, noise_scalings, plot=False, Ag=0, reflection=False):
        
    od = get_tepcat_planets()

    od['g'] = calc_surface_g(od['M_b']*cs.M_jup.value, od['R_b']*cs.R_jup.value)
    od['Ag'] = np.ones(len(od['System']))*Ag
    od['TEQ'] = calc_eq_temp(od['a(AU)']*cs.au.value, od['Teff'], od['R_A']*cs.R_sun.value, A=Ag)

    od['EDE'] = eclipse_depth(od['TEQ'], od['Teff'], od['R_b']*cs.R_jup.value, od['R_A']*cs.R_sun.value, band=band)
    od['EDR'] = reflected_depth(od['R_b']*cs.R_jup.value, od['a(AU)']*cs.au.value, A=Ag)
    od['ED'] = od['EDE'] + od['EDR']
        
    scale = noise_scalings[instrument]['Eclipse']
    scale_ind = list(od['System']).index(scale['System'])
    scale_N = scale['Noise']
    scale_mag = od[scale_band][scale_ind]
    
    
    od['EESNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDE'] / scale_N
    od['ERSNR'] =  (10**(-(od[scale_band]-scale_mag)/2.5/2.))*od['EDR'] / scale_N
    od['ESNR'] =   (10**(-(od[scale_band]-scale_mag)/2.5/2.))*(od['ED']) / scale_N
    od['E_N'] =  scale_N / (10**(-(od[scale_band]-scale_mag)/2.5/2.))*1e6
        
    od['EDE'] *= 1e6; od['EDR'] *= 1e6; od['ED'] *= 1e6
    table_name = './tables/{}/emission_snr_{}_A-{}.csv'.format(instrument, scale_band[0], Ag)
    csv_arrays(table_name, od, 
       keys=['System', scale_band, 'EDE', 'EDR', 'E_N', 'Ag', 'TEQ', 'M_b'],
       names=['Name', scale_band, 'Depth (Em)', 'Depth (Rf)', 'Noise', 'Albedo', 'Eq Temp', 'Mass'],
       units = ['-', '-', 'ppm', 'ppm', 'ppm', '-', 'K', 'M_jup'],
       formats=['{}','{:.2f}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.2f}', '{:.0f}', '{:.2f}'],
       sort='ESNR', reverse=True, finite=True)
    if plot:
        afile = open(table_name, 'r+')
        csvReader1 = csv.reader(afile)
        lines = [ csvReader1.next() for _ in range(22) ]
        for line in lines:
            for val in line:
                print '{:<10s}'.format(val),
            print 
    
    ymax = np.max(od['ED'][np.isfinite(od['ED'])])
    plot_html(od, xname='M_b', yname='ED', x_range=(-0.1,13), y_range=(0,ymax), 
        title='Eclipse depth, {}-band with {}, Ag={}'.format(scale_band[0], instrument, Ag), 
        fname='./tables/{}/eclipse_snr_{}_A-{}.html'.format(instrument, scale_band[0], Ag), plot=plot)
    
    return od

In [41]:
od = get_emission_band('Spitzer', (4e-6,5e-6), 'Kmag', noise_scalings, plot=True)

Name       Kmag       Depth (Em) Depth (Rf) Noise      Albedo     Eq Temp    Mass      
-          -          ppm        ppm        ppm        -          K          M_jup     
NLTT_41135 -1.00      1143.3     0.0        1.5        0.00       548        33.70     
HD_189733  5.54       2503.9     0.0        30.3       0.00       1417       1.15      
WASP-033   7.47       4174.9     0.0        73.8       0.00       3252       2.16      
HD_209458  6.31       1893.8     0.0        43.3       0.00       1735       0.71      
KELT-09    7.48       2643.5     0.0        74.1       0.00       4817       2.88      
KELT-20    7.42       2497.6     0.0        72.1       0.00       2687       0.00      
WASP-076   8.24       3221.3     0.0        105.2      0.00       2595       0.92      
WASP-018   8.13       3051.0     0.0        100.0      0.00       2868       10.52     
WASP-077   8.40       3352.5     0.0        113.2      0.00       2028       1.76      
WASP-121   9.37       5161.4    